In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

INSTAGRAM: 

RDD usuarios = (id_usuario, nickname, pais, fecha, alta)

RDD publiaciones = (id_publicacion, id_usario, id_foto, comentario, fecha)

Buscar top 10 # hashtags usarios argentina, si no fue publicado por alguien de argentina no considerarlo.

In [2]:
usuarios = [(1, 'user1', 'AR', 'fecha', 'alta'),
          (2, 'user2', 'UY', 'fecha', 'alta'),
          (3, 'user3', 'AR', 'fecha', 'alta'),
          (4, 'user4', 'UY', 'fecha', 'alta'),
          (5, 'user5', 'AR', 'fecha', 'alta')]

publicaciones = [(91, 1, 241, '#buendia #telefe, #noticiero', 'fecha'),
                 (92, 1, 121, '#buendia salio el sol', 'fecha'),
                 (93, 1,  234, 'sin filtro', 'fecha'),
                 (103, 1, 239, 'en el cine #endgame', 'fecha'),
                 (94, 2,  432, '#buendia #endgame', 'fecha'),
                 (95, 2, 431, '#endgame #marvel peliculon!', 'fecha'),
                 (96, 3, 213, '#telefe', 'fecha'),
                 (97, 3, 455, '#endgame #marvel', 'fecha'),
                 (98, 4,  123, '#endgame wow #marvel', 'fecha'),
                 (99, 4, 523, '#buendia#holamundo#hacking#fiuy', 'fecha'),
                 (100, 4, 341, 'con amigos', 'fecha'),
                 (100, 4, 341, 'con amigos #cine#marvel#endgame', 'fecha'),
                 (101, 5,  326, 'desayunando cafe, #buendia', 'fecha')]


In [3]:
rdd_usuarios = sc.parallelize(usuarios)
rdd_publicaciones = sc.parallelize(publicaciones)

In [10]:
user_country = rdd_usuarios.filter(lambda x: x[2] == 'AR').\
                map(lambda x: (x[0],x[2]))
user_country.collect()

[(1, 'AR'), (3, 'AR'), (5, 'AR')]

In [5]:
user_coment = rdd_publicaciones.map(lambda x: (x[1], x[3]))
user_coment.collect()

[(1, '#buendia #telefe, #noticiero'),
 (1, '#buendia salio el sol'),
 (1, 'sin filtro'),
 (1, 'en el cine #endgame'),
 (2, '#buendia #endgame'),
 (2, '#endgame #marvel peliculon!'),
 (3, '#telefe'),
 (3, '#endgame #marvel'),
 (4, '#endgame wow #marvel'),
 (4, '#buendia#holamundo#hacking#fiuy'),
 (4, 'con amigos'),
 (4, 'con amigos #cine#marvel#endgame'),
 (5, 'desayunando cafe, #buendia')]

In [11]:
join_a = user_coment.join(user_country)
join_a.collect()

[(1, ('#buendia #telefe, #noticiero', 'AR')),
 (1, ('#buendia salio el sol', 'AR')),
 (1, ('sin filtro', 'AR')),
 (1, ('en el cine #endgame', 'AR')),
 (3, ('#telefe', 'AR')),
 (3, ('#endgame #marvel', 'AR')),
 (5, ('desayunando cafe, #buendia', 'AR'))]

In [62]:
coments = join_a.map(lambda x: x[1][0])
coments.collect()

['#buendia #telefe, #noticiero',
 '#buendia salio el sol',
 'sin filtro',
 'en el cine #endgame',
 '#telefe',
 '#endgame #marvel',
 'desayunando cafe, #buendia']

In [88]:
top_hashtag = coments.flatMap(lambda x: x.split()).\
            filter(lambda x: x[0] =="#").\
            map(lambda x: (x,1)).\
            reduceByKey(lambda x,y: x+y).\
            sortByKey().take(1)
top_hashtag

[('#buendia', 3)]